<a href="https://colab.research.google.com/github/annvorosh/GB/blob/MLB_FinalProject/MLB_L09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Домашнее задание

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

По шагам:
0. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
1. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
3. завести себе аккаунт на github (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
4. итоговый проект должен содержать: 1) каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна) 2) файл app/run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта) 4) README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д) 5) Dockerfile 6) docker-entrypoint.sh
5. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

Полезные ссылки:
1. датасеты (для полета мысли): https://www.kaggle.com/datasets
2. конкурс Сбербанка по недвижимости (можно этот набор данных также взять и обучить модель предсказывать стоимость жилья - неплохой такой сервис может получиться) - https://www.kaggle.com/c/sberbank-russian-housing-market/data Там же и ноутбуки с разными подходами есть.
3. минималистичный пример связки keras/flask https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html для определения класса картинки
4. неплохой такой пример (помимо того, что разобрали на занятии) связки docker/flask - https://cloud.croc.ru/blog/byt-v-teme/flask-prilozheniya-v-docker/
5. https://www.digitalocean.com/community/tutorials/how-to-build-and-deploy-a-flask-application-using-docker-on-ubuntu-18-04

p.s. если проблемы с выбором датасета, то пишите пожалуйста - будем вместе думать)

## Step 1 - TRAIN

### Обучение пайплайна

- Загрузим данные https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction
- Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
- Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [1]:
# pip freeze > requirements.txt

In [2]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

### Загрузим датасет
https://www.kaggle.com/datasets/deepcontractor/200k-short-texts-for-humor-detection

- 200K SHORT TEXTS FOR HUMOR DETECTION
- Classify if a text sentence is funny or not



In [3]:
import pandas as pd

df = pd.read_csv('dataset.csv')


In [4]:
df.head()


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [5]:
df['humor'].value_counts()

False    100000
True     100000
Name: humor, dtype: int64

In [6]:
# df['humor'] = df['humor'].replace({False: 0, True: 1})


In [7]:
# Разделим данные на train/test и сохраним тестовую выборку на диск
X_train, X_test, y_train, y_test = train_test_split(df, df['humor'], test_size=0.33, random_state=42, stratify=df['humor'])
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [8]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [9]:
features = ['text']
target = 'humor'

Соберем кусок, ответственный за feature engineering

In [10]:
X_train.head()

,text,humor
190341,90% of the economy is just women giving each o...,True
181885,Maybe don't show me a picture if you don't wan...,True
102920,From surviving a hate crime to becoming dads: ...,False
165155,How can you tell an auto mechanic just had sex...,True
2326,"Chuck norris' house has no doors, only walls t...",True


In [11]:
# combine
text = Pipeline([
                ('imputer', TextImputer('text', '')),
                ('selector', ColumnSelector(key='text')),
                ('tfidf', TfidfVectorizer())
            ])


feats = FeatureUnion([('text', text)])

In [12]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 7.99 s, sys: 5.08 s, total: 13.1 s
Wall time: 13.8 s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='text',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [13]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('text',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='text',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='text')),
                                                  ('tfidf',
                                                   TfidfVectorizer())]))])),
 ('classifier', LogisticRegression())]

Сохраним модель (пайплайн)

In [14]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

## Step 2 - PREDICT
### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [15]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [16]:
X_test.head(3)

,text,humor
0,Neighborhood full of million-dollar homes is n...,False
1,Life with 'devourers' and 'baby/bullies' -- yo...,False
2,Brazil's miss bumbum stirs controversy with bo...,False


In [17]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [18]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='text',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [19]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [20]:
preds[:10]

array([0.50727654, 0.17659506, 0.03947028, 0.4562907 , 0.25830758,
       0.01790641, 0.27613949, 0.27790571, 0.95623218, 0.83246921])

In [21]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.475954228695647, F-Score=0.924, Precision=0.920, Recall=0.928


## Step 3 - FLASK

### При внедрении
При внедрении необходимо сделать:
- Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
- Определить ip-адрес и порт, на который будут поступать данные.
- Создать во Flask необходимые роуты:
  * @app.route('/predict_example', method='POST')
  * def predict_example():
- Перенести во Flask все функции преобразования данных,
формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
- Загрузить обученные модели.
- Настроить логирование.
### Flask
Тут будет сервис для обработки запросов на Flask
Google Colab предоставляет виртуальную машину, поэтому мы не можем получить доступ к локальному хосту, как это делаем на нашем локальном компьютере при запуске локального веб-сервера. Что мы можем сделать, так это предоставить его общедоступному URL-адресу с помощью ngrok.
https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [22]:
!pip install flask-ngrok

In [23]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [24]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2S6NSKVUOqUpXjR65430IsMgYPx_6Rvd9k2tf8ZpU12VvMMod # https://dashboard.ngrok.com/get-started/setup
!./ngrok http 5000

--2023-07-04 15:02:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.1’

ngrok-stable-linux- 100%[===================>]  13.21M  54.2MB/s    in 0.2s    

2023-07-04 15:02:45 (54.2 MB/s) - ‘ngrok-stable-linux-amd64.tgz.1’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### **Создаем сервис для обработки запросов к модели**

In [25]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [26]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [27]:
# Запустить сервис и не глушить его, пока работаем

In [28]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    text = ""
    request_json = request.get_json()

    if request_json["text"]:
        text = request_json['text']


    print(text)
    preds = model.predict_proba(pd.DataFrame({"text": [text]}))
    data["predictions"] = preds[:, 1][0]
    data["text"] = text
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e87e-34-138-8-49.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [04/Jul/2023 15:06:32] "POST /predict HTTP/1.1" 200 -


We are negotiable on salary and there is the potential for equity for the right candidate.
OK


...

{"text": "What is the room temperature on tatooine? lukewarm"}
OK
{"text": "There's this band called one thousand and twenty three megabytes they haven't had any gigs yet."}
OK
{"text": "Holiday roast: slow roasted pork shoulder -- pear, ginger chutney"}
OK


INFO:werkzeug:127.0.0.1 - - [04/Jul/2023 15:54:46] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2023 15:54:46] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2023 15:54:46] "POST /predict HTTP/1.1" 200 -


{"text": "A 'real' school of wizardry is opening in an english castle this year"}
OK
{"text": "There's a cool sport called volcano diving..... you'll only do it once."}
OK
{"text": "Where is the world's first mainstream transgender superhero?"}
OK


INFO:werkzeug:127.0.0.1 - - [04/Jul/2023 15:54:46] "POST /predict HTTP/1.1" 200 -


{"text": "Pope francis will visit three african countries in november"}
OK
